<img src="./images/DLI_Header.png" style="width: 400px;">

# Assessment

Congratulations on going through today's course! Hopefully, you've learned some valuable skills along the way and had fun doing it. Now it's time to put those skills to the test. In this assessment, you will train a new model that is able to recognize fresh and rotten fruit. You will need to get the model to a validation accuracy of `92%` in order to pass the assessment, though we challenge you to do even better if you can. You will have the use the skills that you learned in the previous exercises. Specifically, we suggest using some combination of transfer learning, data augmentation, and fine tuning. Once you have trained the model to be at least 92% accurate on the validation dataset, save your model, and then assess its accuracy. Let's get started! 

## The Dataset

In this exercise, you will train a model to recognize fresh and rotten fruits. The dataset comes from [Kaggle](https://www.kaggle.com/sriramr/fruits-fresh-and-rotten-for-classification), a great place to go if you're interested in starting a project after this class. The dataset structure is in the `data/fruits` folder. There are 6 categories of fruits: fresh apples, fresh oranges, fresh bananas, rotten apples, rotten oranges, and rotten bananas. This will mean that your model will require an output layer of 6 neurons to do the categorization successfully. You'll also need to compile the model with `categorical_crossentropy`, as we have more than two categories.

<img src="./images/fruits.png" style="width: 600px;">

## Load ImageNet Base Model

We encourage you to start with a model pretrained on ImageNet. Load the model with the correct weights, set an input shape, and choose to remove the last layers of the model. Remember that images have three dimensions: a height, and width, and a number of channels. Because these pictures are in color, there will be three channels for red, green, and blue. We've filled in the input shape for you. This cannot be changed or the assessment will fail. If you need a reference for setting up the pretrained model, please take a look at [notebook 05b](05b_presidential_doggy_door.ipynb) where we implemented transfer learning.

In [1]:
from tensorflow import keras


base_model = keras.applications.VGG16(
    weights='imagenet',  # Load weights pre-trained on ImageNet.
    input_shape=(224, 224, 3),
    include_top=False)

## Freeze Base Model

Next, we suggest freezing the base model, as done in [notebook 05b](05b_presidential_doggy_door.ipynb). This is done so that all the learning from the ImageNet dataset does not get destroyed in the initial training.

In [2]:
# Freeze base model
base_model.trainable = False

## Add Layers to Model

Now it's time to add layers to the pretrained model. [Notebook 05b](05b_presidential_doggy_door.ipynb) can be used as a guide. Pay close attention to the last dense layer and make sure it has the correct number of neurons to classify the different types of fruit.

In [3]:


inputs = keras.Input(shape=(224, 224, 3))
# Separately from setting trainable on the model, we set training to False 
x = base_model(inputs, training=False)
x = keras.layers.GlobalAveragePooling2D()(x)

outputs = keras.layers.Dense(6)(x)
model = keras.Model(inputs, outputs)

In [4]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
vgg16 (Model)                (None, 7, 7, 512)         14714688  
_________________________________________________________________
global_average_pooling2d (Gl (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 6)                 3078      
Total params: 14,717,766
Trainable params: 3,078
Non-trainable params: 14,714,688
_________________________________________________________________


## Compile Model

Now it's time to compile the model with loss and metrics options. Remember that we're training on a number of different categories, rather than a binary classification problem.

In [6]:
model.compile(loss=keras.losses.CategoricalCrossentropy(from_logits=True), metrics=[keras.metrics.CategoricalAccuracy()])

## Augment the Data

If you'd like, try to augment the data to improve the dataset. Feel free to look at [notebook 04a](04a_asl_augmentation.ipynb) and [notebook 05b](05b_presidential_doggy_door.ipynb) for augmentation examples. There is also documentation for the [Keras ImageDataGenerator class](https://keras.io/api/preprocessing/image/#imagedatagenerator-class). This step is optional, but it may be helpful to get to 92% accuracy.

In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
        samplewise_center=True,  # set each sample mean to 0
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False) # we don't expect Bo to be upside-down so we will not flip vertically

## Load Dataset

Now it's time to load the train and validation datasets. Pick the right folders, as well as the right `target_size` of the images (it needs to match the height and width input of the model you've created). If you'd like a reference, you can check out [notebook 05b](05b_presidential_doggy_door.ipynb).

In [8]:

# load and iterate training dataset
train_it = datagen.flow_from_directory('data/fruits/train/', 
                                       target_size=(224, 224), 
                                       color_mode='rgb', 
                                       class_mode='categorical', 
                                       batch_size=8)
# load and iterate validation dataset
valid_it = datagen.flow_from_directory('data/fruits/valid/', 
                                      target_size=(224, 224), 
                                      color_mode='rgb', 
                                      class_mode='categorical', 
                                      batch_size=8)

Found 1182 images belonging to 6 classes.
Found 329 images belonging to 6 classes.


## Train the Model

Time to train the model! Pass the `train` and `valid` iterators into the `fit` function, as well as setting your desired number of epochs.

In [9]:
model.fit(train_it, steps_per_epoch=12, validation_data=valid_it, validation_steps=4, epochs=20)

Epoch 1/20
12/12 [==============================] - 4s 333ms/step - loss: 6.5561 - categorical_accuracy: 0.1875 - val_loss: 4.1432 - val_categorical_accuracy: 0.1562
Epoch 2/20
12/12 [==============================] - 2s 149ms/step - loss: 2.8279 - categorical_accuracy: 0.3958 - val_loss: 2.9097 - val_categorical_accuracy: 0.5000
Epoch 3/20
12/12 [==============================] - 2s 149ms/step - loss: 2.6547 - categorical_accuracy: 0.4479 - val_loss: 1.9396 - val_categorical_accuracy: 0.5000
Epoch 4/20
12/12 [==============================] - 2s 152ms/step - loss: 1.9132 - categorical_accuracy: 0.6146 - val_loss: 1.1009 - val_categorical_accuracy: 0.7188
Epoch 5/20
12/12 [==============================] - 2s 149ms/step - loss: 1.5076 - categorical_accuracy: 0.6354 - val_loss: 0.8505 - val_categorical_accuracy: 0.7500
Epoch 6/20
12/12 [==============================] - 2s 144ms/step - loss: 1.7209 - categorical_accuracy: 0.5625 - val_loss: 0.6409 - val_categorical_accuracy: 0.7500
Epoc

## Unfreeze Model for Fine Tuning

If you have reached 92% validation accuracy already, this next step is optional. If not, we suggest fine tuning the model with a very low learning rate.

In [10]:
# Unfreeze the base model
base_model.trainable = True

# It's important to recompile your model after you make any changes
# to the `trainable` attribute of any inner layer, so that your changes
# are taken into account
model.compile(optimizer=keras.optimizers.RMSprop(learning_rate = .00001),  # Very low learning rate
              loss=keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=[keras.metrics.CategoricalAccuracy()])

In [11]:
model.fit(train_it, steps_per_epoch=12, validation_data=valid_it, validation_steps=4, epochs=10)

Epoch 1/10
12/12 [==============================] - 8s 640ms/step - loss: 0.4858 - categorical_accuracy: 0.8750 - val_loss: 0.1595 - val_categorical_accuracy: 0.9688
Epoch 2/10
12/12 [==============================] - 2s 167ms/step - loss: 0.3054 - categorical_accuracy: 0.8750 - val_loss: 0.1124 - val_categorical_accuracy: 0.9375
Epoch 3/10
12/12 [==============================] - 2s 169ms/step - loss: 0.4296 - categorical_accuracy: 0.9167 - val_loss: 0.2169 - val_categorical_accuracy: 0.9062
Epoch 4/10
12/12 [==============================] - 2s 168ms/step - loss: 0.1494 - categorical_accuracy: 0.9375 - val_loss: 0.0950 - val_categorical_accuracy: 0.9375
Epoch 5/10
12/12 [==============================] - 3s 275ms/step - loss: 0.5088 - categorical_accuracy: 0.8830 - val_loss: 0.5259 - val_categorical_accuracy: 0.9062
Epoch 6/10
12/12 [==============================] - 2s 161ms/step - loss: 0.2230 - categorical_accuracy: 0.8958 - val_loss: 0.2449 - val_categorical_accuracy: 0.9200
Epoc

## Evaluate the Model

Hopefully, you now have a model that has a validation accuracy of 92% or higher. If not, you may want to go back and either run more epochs of training, or adjust your data augmentation. 

Once you are satisfied with the validation accuracy, evaluate the model by executing the following cell. The evaluate function will return a tuple, where the first value is your loss, and the second value is your accuracy. To pass, the model will need have an accuracy value of `92% or higher`. 

In [12]:
model.evaluate(valid_it, steps=valid_it.samples/valid_it.batch_size)

42/41 [==============================] - 4s 101ms/step - loss: 0.1341 - categorical_accuracy: 0.9635


[0.13408349454402924, 0.9635258316993713]

## Run the Assessment

To assess your model run the following two cells.

**NOTE:** `run_assessment` assumes your model is named `model` and your validation data iterator is called `valid_it`. If for any reason you have modified these variable names, please update the names of the arguments passed to `run_assessment`.

In [13]:
from run_assessment import run_assessment

In [14]:
run_assessment(model, valid_it)

Evaluating model 5 times to obtain average accuracy...

42/41 [==============================] - 4s 102ms/step - loss: 0.1284 - categorical_accuracy: 0.9605

Accuracy required to pass the assessment is 0.92 or greater.
Your average accuracy is 0.9544.

Congratulations! You passed the assessment!
See instructions below to generate a certificate.
